<a href="https://colab.research.google.com/github/msmsm104/MachineLearning_Project/blob/main/Definition/feature_engineering_4%EC%B0%A8(0613).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 4차 전처리 


In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt  

from timeit import default_timer as timer
from sklearn import preprocessing

# !pip install ultimate
# from ultimate.mlp import MLP 

import gc, sys
gc.enable()


In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
INPUT_DIR = '/content/drive/MyDrive/Colab Notebooks/이어드림스쿨/TIL(Today I Learned)/20220607/ML-project/data/Interim/'

In [ ]:
def state(message,start = True, time = 0):
    if(start):
        print(f'Working on {message} ... ')
    else :
        print(f'Working on {message} took ({round(time , 3)}) Sec \n')

In [ ]:
def feature_engineering(is_train=True):
    if is_train: 
        print("processing train_V2.csv")
        df = pd.read_csv(INPUT_DIR + 'train_V2.csv')
        df = df[df['maxPlace'] > 1]
    else:
        print("processing test_V2.csv")
        df = pd.read_csv(INPUT_DIR + 'test_V2.csv')

    state('rankPoints')
    s = timer()
    # Process the 'rankPoints' feature by replacing any value of (-1) to be (0) :
    df['rankPoints'] = np.where(df['rankPoints'] <= 0 ,0 , df['rankPoints'])
    e = timer()                                  
    state('rankPoints', False, e-s)


    target = 'winPlacePerc'
    # Get a list of the features to be used
    features = list(df.columns)
    
    # Remove some features from the features list :
    features.remove("Id")
    features.remove("matchId")
    features.remove("groupId")
    features.remove("matchDuration")
    features.remove("matchType")
    
    y = None

    if is_train: 
        y = np.array(df.groupby(['matchId','groupId'])[target].agg('mean'), dtype=np.float64)
        # Remove the target from the features list :
        features.remove(target)

    print("get group mean feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('mean')
    # Put the new features into a rank form ( max value will have the highest rank)
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()


    if is_train: 
        df_out = agg.reset_index()[['matchId','groupId']]
    # If we are processing the test data let df_out = 'matchId' and 'groupId' without grouping 
    else: 
        df_out = df[['matchId','groupId']]

    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_mean", "_mean_rank"], how='left', on=['matchId', 'groupId'])

    df_out.drop(["matchId", "groupId"], axis=1, inplace=True)

    X = np.array(df_out, dtype=np.float64)

    del df, agg, agg_rank
    gc.collect()

    df_out['winPlacePerc'] = y

    return df_out


In [ ]:
data = feature_engineering()

processing train_V2.csv
Working on rankPoints ... 
Working on rankPoints took (0.075) Sec 



In [ ]:
data.head(5)

,assists_mean,boosts_mean,damageDealt_mean,DBNOs_mean,headshotKills_mean,heals_mean,killPlace_mean,killPoints_mean,kills_mean,killStreaks_mean,...,revives_mean_rank,rideDistance_mean_rank,roadKills_mean_rank,swimDistance_mean_rank,teamKills_mean_rank,vehicleDestroys_mean_rank,walkDistance_mean_rank,weaponsAcquired_mean_rank,winPoints_mean_rank,winPlacePerc
0,0.0,0.5,109.675000,1.000000,0.0,0.5,41.0,1242.0,1.0,0.5,...,0.339286,0.375000,0.517857,0.410714,0.5,0.482143,0.285714,0.160714,0.357143,0.3333
1,0.0,0.0,47.988333,0.333333,0.0,0.0,90.5,1355.5,0.0,0.0,...,0.339286,0.375000,0.517857,0.410714,0.5,0.482143,0.071429,0.107143,0.142857,0.0370
2,0.0,0.0,0.000000,0.000000,0.0,0.0,94.5,1382.0,0.0,0.0,...,0.339286,0.375000,0.517857,0.410714,0.5,0.482143,0.035714,0.035714,0.571429,0.0000
3,0.0,0.5,11.700000,0.000000,0.0,0.0,59.5,1178.0,0.0,0.0,...,0.339286,0.375000,0.517857,0.410714,0.5,0.482143,0.392857,1.000000,0.107143,0.3704
4,1.0,3.5,340.950000,2.500000,1.0,1.0,14.0,1504.0,3.0,1.5,...,0.339286,0.821429,0.517857,0.892857,0.5,0.482143,0.964286,0.625000,0.785714,1.0000


In [ ]:
data.to_csv(INPUT_DIR + 'featured_train_4.csv', index=False)

In [ ]:
reduce_mem_usage(pd.read_csv(INPUT_DIR + 'featured_train_4.csv'))

Memory usage of dataframe is 726.75 MB
Memory usage after optimization is: 181.69 MB
Decreased by 75.0%


,assists_mean,boosts_mean,damageDealt_mean,DBNOs_mean,headshotKills_mean,heals_mean,killPlace_mean,killPoints_mean,kills_mean,killStreaks_mean,...,revives_mean_rank,rideDistance_mean_rank,roadKills_mean_rank,swimDistance_mean_rank,teamKills_mean_rank,vehicleDestroys_mean_rank,walkDistance_mean_rank,weaponsAcquired_mean_rank,winPoints_mean_rank,winPlacePerc
0,0.000000,0.500000,109.687500,1.000000,0.000000,0.500000,41.000000,1242.0,1.000000,0.500000,...,0.339355,0.375000,0.518066,0.410645,0.500000,0.482178,0.285645,0.160767,0.357178,0.333252
1,0.000000,0.000000,48.000000,0.333252,0.000000,0.000000,90.500000,1356.0,0.000000,0.000000,...,0.339355,0.375000,0.518066,0.410645,0.500000,0.482178,0.071411,0.107117,0.142822,0.036987
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,94.500000,1382.0,0.000000,0.000000,...,0.339355,0.375000,0.518066,0.410645,0.500000,0.482178,0.035706,0.035706,0.571289,0.000000
3,0.000000,0.500000,11.703125,0.000000,0.000000,0.000000,59.500000,1178.0,0.000000,0.000000,...,0.339355,0.375000,0.518066,0.410645,0.500000,0.482178,0.392822,1.000000,0.107117,0.370361
4,1.000000,3.500000,341.000000,2.500000,1.000000,1.000000,14.000000,1504.0,3.000000,1.500000,...,0.339355,0.821289,0.518066,0.893066,0.500000,0.482178,0.964355,0.625000,0.785645,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026739,0.000000,0.000000,16.953125,0.000000,0.000000,0.000000,29.000000,0.0,1.000000,1.000000,...,0.362061,0.724121,0.482666,0.482666,0.482666,0.500000,0.896484,0.413818,0.517090,0.643066
2026740,0.666504,2.666016,205.625000,1.333008,0.666504,6.667969,19.328125,0.0,1.333008,1.000000,...,1.000000,0.931152,0.965332,0.482666,0.482666,0.500000,0.689453,0.844727,0.517090,0.928711
2026741,0.000000,0.000000,25.953125,0.166626,0.000000,0.000000,82.000000,0.0,0.166626,0.166626,...,0.362061,0.258545,0.482666,0.482666,0.482666,0.500000,0.034485,0.068970,0.517090,0.000000
2026742,0.000000,0.500000,59.750000,0.750000,0.000000,0.500000,60.750000,0.0,0.250000,0.250000,...,0.362061,0.258545,0.482666,0.482666,0.482666,0.500000,0.206909,0.344727,0.517090,0.250000
